In [1]:
import scipy.io as sio
import os
import gzip
import nibabel as nib
import numpy as np

from dipy.core.gradients import gradient_table
from dipy.data import get_sphere
from dipy.direction import peaks_from_model
from scipy.io.matlab import loadmat
from dipy.io import read_bvals_bvecs
from dipy.reconst.shm import sf_to_sh, sph_harm_lookup, smooth_pinv
from dipy.reconst.gqi import GeneralizedQSamplingModel

#Importing the dsi studio fib file

In [2]:
fib_file = ('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/dsi_studio/data_con.nii.gz.src.gz.odf8.f5rec.bal.012fy.rdi.gqi.1.25.fib.gz')
f = gzip.open(fib_file, 'r') 
m = loadmat(f)

In [4]:
odfs_file = ('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/dsi_studio/odfs.mat')
odfs = loadmat (odfs_file)

odf_full_file = ('/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/dsi_studio/odf_full.mat')
odf_full = loadmat (odf_full_file)

In [5]:
odfs_ar = odfs['odfs']
odfs_full_ar = odf_full['odf_full']
odfs_ar.shape
odfs_full_ar.shape
vertices = m['odf_vertices']
vertices.shape

(3, 642)

In [6]:
vertices

array([[ 0.        ,  0.04262959, -0.11160571, ...,  0.30901703,
        -0.80901694,  0.809017  ],
       [ 0.        ,  0.13120039,  0.0810863 , ..., -0.95105654,
        -0.5877853 , -0.58778524],
       [ 1.        ,  0.99043888,  0.99043888, ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [102]:
data.shape
shape = data.shape[:-1]  #3d
odf_array = np.zeros((shape + (len(odfs_ar),)))
odf_array.shape

(140, 140, 96, 321)

In [108]:
from dipy.core.ndindex import ndindex

for idx in ndindex(odfs_full):
    odf_array[...3] = odfs_ar[idx]
    odf_ar[idx]=odf_fin

TypeError: only integer scalar arrays can be converted to a scalar index

In [53]:
ndindex(shape)

<generator object ndindex at 0x2b467840ce60>

In [5]:
odf0 = m['odf0']
odf1 = m['odf1']
print("odf0 shape: " + str(odf0.shape))
print("odf1 shape: " + str(odf1.shape))
print (odf0.dtype)
vertices = m['odf_vertices']
vertices.shape

odf0 shape: (321, 20000)
odf1 shape: (321, 20000)
float32


(3, 642)

In [19]:
fimg = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/data.nii.gz")
img = nib.load(fimg)
data = img.get_data()

fbval=("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/bval.txt")
fbvec = ("/space/hemera/1/users/cmaffei/scilpy_mrtrix_comparison/1000/from_mgh/bvec.txt")
bvals, bvecs = read_bvals_bvecs(fbval, fbvec)
gtab = gradient_table(bvals,bvecs)

In [8]:
sphere = get_sphere('symmetric724')
gqmodel = GeneralizedQSamplingModel(gtab, sampling_length=1.2)  #sampling length is lambda
gqpeaks = peaks_from_model(model=gqmodel,
                           data=data,
                           sphere=sphere,
                           relative_peak_threshold=.35,         #try .35 as in the test
                           min_separation_angle=25, return_odf=True,
                           normalize_peaks=True)

gqpeak_values = gqpeaks.peak_values    #value of odf at a maximum point. odf[peak_indices[i]]==peak_values[i]
gqpeak_indices = gqpeaks.peak_indices  #show which sphere points have the maximum values


In [12]:
x = gqpeaks.odf[30, 30 , 30, :]
x

array([ 16.81981725,  19.11920691,  16.80020708,  17.13030803,
        18.88827719,  15.89952295,  20.56300853,  17.26903065,
        15.21667883,  21.58651524,  17.1540673 ,  18.30139436,
        18.14985067,  15.0722863 ,  23.80603586,  17.73623599,
        15.22694328,  20.82742841,  17.63441956,  21.24722356,
        17.639847  ,  14.19205868,  25.23506957,  19.39671326,
        16.24696287,  18.93222939,  16.66800807,  25.49293184,
        18.31821031,  14.19057022,  23.44371924,  21.1130166 ,
        18.81685704,  18.09454989,  14.82892778,  28.31720338,
        20.53556888,  14.67354017,  20.26432602,  20.5255531 ,
        23.98356755,  18.50092401,  14.26384959,  26.89226505,
        23.96105123,  15.62338103,  18.69310931,  17.39823761,
        29.45046569,  20.18166399,  14.73654463,  22.52627159,
        25.44117999,  19.73002578,  18.92491522,  15.15580591,
        30.35329814,  24.38858721,  14.22171601,  19.4629014 ,
        22.27179972,  27.27299797,  19.57688383,  15.65

In [ ]:
sh_gqi = sf_to_sh(gqfit.odf(sphere), sphere, sh_order=6, basis_type='mrtrix', smooth=0.0)
nib.save(nib.Nifti1Image(sh_gqi.astype(np.float32), img.affine, img.header), 'gqi_odf_sf2sh.nii.gz')